In [47]:
import pandas as pd
digital_class=pd.read_csv(r'C:\Users\Russhi\Desktop\BOOTCAMP\Module3_0_ProyectoFinal\digital_class.csv')
digital_class.head()

,prices.amountMax,prices.amountMin,Disponibilidad,Estado,prices.isSale,Tienda Precio,Envios,Categoria Principal,secondary category
0,104.99,104.99,In Stock,New,0,bestbuy,Free Standard Shipping,TV and accessories,TV
1,69.00,64.99,In Stock,New,1,walmart,Free Expedited Shipping,TV and accessories,Home Theater
2,69.00,69.00,In Stock,New,0,walmart,Free Expedited Shipping,TV and accessories,Home Theater
3,69.99,69.99,In Stock,New,0,bestbuy,Free Standard Shipping,TV and accessories,Home Theater
4,66.99,66.99,In Stock,New,0,bestbuy,Free Standard Shipping,TV and accessories,Home Theater


In [48]:
from sklearn.preprocessing import OneHotEncoder
encoder= OneHotEncoder(drop='first', sparse_output=False)
columnas=['Disponibilidad', 'Estado', 'Envios','Categoria Principal','secondary category']
categorical_data = digital_class[columnas]
encoded_categorical_data = pd.DataFrame(encoder.fit_transform(categorical_data))
encoded_categorical_data.columns = encoder.get_feature_names_out(columnas)
digital_class = digital_class.drop(['Disponibilidad', 'Estado', 'Envios','Categoria Principal','secondary category'], axis=1)
encoded_data = pd.concat([digital_class, encoded_categorical_data], axis=1)

In [49]:
encoded_data['Tienda Precio']=encoded_data['Tienda Precio'].map({'bestbuy':0,'walmart':1,'bhphotovideo':2,'ebay':3})

In [50]:
from sklearn.model_selection import train_test_split
X = encoded_data.drop(columns = ['Tienda Precio'], axis=1).copy() 
y = encoded_data['Tienda Precio']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state=357)

In [51]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [52]:
from tensorflow.keras.utils import to_categorical
# Codificación OneHotEncoder para las variables categóricas de salida.
y_train=to_categorical(y_train)
y_test=to_categorical(y_test)

In [53]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, regularizers
# Conjunto lineal de capas en el que la salida de cada capa se convierte en la entrada de la siguiente.
model = models.Sequential([
    # Capa de entrada con 128 filtros, método de activación relu, que introduce no linealidad en el modelo y es la más
    # usada para input y hidden layers. La no linealidad introducida permite al modelo aprender y representar relaciones
    # complejas entre los datos de entrada y salida. Regularización L2, Ridge regularization or weight decay, esto añade
    # una penalización a la función de pérdida, haciendo que el modelo no seleccione pesos muy grandes, de manera que hace
    # que no se apoye en un determinado valor. El input shape son los datos de entrada, que viene definido por las diferentes
    # características/columnas dentro de X_train.
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(X_train.shape[1],)),
    layers.Dropout(0.2), # Esta capa evita el overfitting. Selecciona un 20% de las unidades de entrada y lo pone a cero.
    # Esto fuerza al modelo a no apoyarse en valores determinaddos.
    # Las siguientes capas, formarían parte de la hidden layer.
    layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
])

In [54]:
# Se añade la capa de salida, que tiene cuatro filtros, uno para cada categoría de salida del modelo.
# Softmax convierte la salida en la probabilidad de que ocurra cada valor, representando la probabilidad de ocurrencia
# the likelihood para cada categoría.
model.add(layers.Dense(4, activation='softmax')) 

In [55]:
# Optimizador Adam, optimizador de gradiente estocástico.
# Función de pérdida, que al minimizar permitiría mejorar la precisión del modelo.
# Las métricas, que evalúa el desempeño del modelo y para este caso queremos evaluar la precisión.
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [64]:
# Entrenamiento del modelo.
# Epochs el número de veces (iteraciones) que se pasan los datos al modelo.
# batch size es el tamaño de datos que se pasan a la vez al modelo, para poder actualizar los pesos.
model.fit(X_train, y_train, epochs=30, batch_size=150)

Epoch 1/30
39/39 [==============================] - 0s 2ms/step - loss: 0.7123 - accuracy: 0.7312
Epoch 2/30
39/39 [==============================] - 0s 2ms/step - loss: 0.7124 - accuracy: 0.7314
Epoch 3/30
39/39 [==============================] - 0s 2ms/step - loss: 0.7151 - accuracy: 0.7268
Epoch 4/30
39/39 [==============================] - 0s 1ms/step - loss: 0.7103 - accuracy: 0.7304
Epoch 5/30
39/39 [==============================] - 0s 1ms/step - loss: 0.7117 - accuracy: 0.7271
Epoch 6/30
39/39 [==============================] - 0s 1ms/step - loss: 0.7085 - accuracy: 0.7342
Epoch 7/30
39/39 [==============================] - 0s 1ms/step - loss: 0.7110 - accuracy: 0.7302
Epoch 8/30
39/39 [==============================] - 0s 1ms/step - loss: 0.7167 - accuracy: 0.7221
Epoch 9/30
39/39 [==============================] - 0s 2ms/step - loss: 0.7072 - accuracy: 0.7330
Epoch 10/30
39/39 [==============================] - 0s 2ms/step - loss: 0.7088 - accuracy: 0.7354
Epoch 11/30
39/39 [

In [65]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc}')

46/46 [==============================] - 0s 643us/step - loss: 0.7164 - accuracy: 0.7228
Test accuracy: 0.7227585911750793


In [66]:
model.save('digital_class.h5')

c:\Users\Russhi\Desktop\BOOTCAMP\venv_analytics\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


* Implementación

In [187]:
from tensorflow.keras.models import load_model
import tensorflow as tf
import numpy as np

max_price = float(input("Precio máximo del artículo visto: "))
min_price = float(input("Precio mínimo del artículo visto: "))
disponibilidad = input("'In Stock', 'Out of Stock', 'Special Order', 'More on the Way','Retired': ")
estado = input("'New', 'Refurbished', 'Used': ")
sale = input("Rebajas: yes or no ")
envio = input("'Free Standard Shipping', 'Free Expedited Shipping', 'Minimum Order Free Shipping', 'Paid Shipping': ")
cat_prim = input("'TV and accessories', 'Computer and accessories', 'Wireless Tech', 'Audio and accessories', 'Phones and accessories', 'Electronics': ")

if cat_prim=='TV and accessories':
    cat_sec = input("'TV', 'Home Theater', 'TV and accessories': ")
elif cat_prim=='Computer and accessories':
    cat_sec = input("'Computer and accessories', 'Computer', 'Discos duros': ")
elif cat_prim=='Electronics':
    cat_sec = input("'Digital camera and accessories', 'Electronics': ")
elif cat_prim=='Audio and accessories':
    cat_sec = input("'Audio and accessories', 'Audio': ")
elif cat_prim=='Phones and accessories':
    cat_sec = input("'Phones and accessories','Smart-tech': ")
else:
    cat_sec = input("'Wireless Tech': ")



def prediccion_tienda(max_price,min_price,disponibilidad,estado,sale,envio,cat_prim,cat_sec):
    data = pd.DataFrame(columns=['prices.amountMax', 'prices.amountMin', 'prices.isSale',
       'Disponibilidad_More on the Way',
       'Disponibilidad_Out of Stock', 'Disponibilidad_Retired',
       'Disponibilidad_Special Order', 'Estado_Refurbished', 'Estado_Used',
       'Envios_Free Standard Shipping', 'Envios_Minimum Order Free Shipping',
       'Envios_Paid Shipping', 'Categoria Principal_Computer and accessories',
       'Categoria Principal_Electronics',
       'Categoria Principal_Phones and accessories',
       'Categoria Principal_TV and accessories',
       'Categoria Principal_Wireless Tech',
       'secondary category_Audio and accessories',
       'secondary category_Computer',
       'secondary category_Computer and accessories',
       'secondary category_Digital camera and accessories',
       'secondary category_Discos duros', 'secondary category_Electronics',
       'secondary category_Home Theater',
       'secondary category_Phones and accessories',
       'secondary category_Smart-tech', 'secondary category_TV',
       'secondary category_TV and accessories',
       'secondary category_Wireless Tech'])
    
    maxp_std=(max_price-1.0)/(6999.99-1.0)
    data.loc[0,'prices.amountMax']= maxp_std * (1-0) + 0
    minp_std=(min_price-1.0)/(5999.99-1.0)
    data.loc[0,'prices.amountMin']= minp_std * (1-0) + 0

    if sale=='yes':
        data.loc[0,'prices.isSale']=1
    else:
        data.loc[0,'prices.isSale']=0

    if disponibilidad=='In Stock':
        data.loc[0,['Disponibilidad_More on the Way','Disponibilidad_Out of Stock', 'Disponibilidad_Retired','Disponibilidad_Special Order']] =[0,0,0,0]
    elif disponibilidad=='Out of Stock':
        data.loc[0,['Disponibilidad_More on the Way','Disponibilidad_Out of Stock', 'Disponibilidad_Retired','Disponibilidad_Special Order']] =[0,1,0,0]
    elif disponibilidad=='Special Order':
        data.loc[0,['Disponibilidad_More on the Way','Disponibilidad_Out of Stock', 'Disponibilidad_Retired','Disponibilidad_Special Order']] =[0,0,0,1]
    elif disponibilidad=='More on the Way':
        data.loc[0,['Disponibilidad_More on the Way','Disponibilidad_Out of Stock', 'Disponibilidad_Retired','Disponibilidad_Special Order']] =[1,0,0,0]
    else:
        data.loc[0,['Disponibilidad_More on the Way','Disponibilidad_Out of Stock', 'Disponibilidad_Retired','Disponibilidad_Special Order']] =[0,0,1,0]

    if estado=='New':
        data.loc[0,['Estado_Refurbished', 'Estado_Used']]=[0,0]
    elif estado=='Refurbished':
        data.loc[0, ['Estado_Refurbished', 'Estado_Used']] = [1, 0]
    else:
        data.loc[0, ['Estado_Refurbished', 'Estado_Used']] = [0, 1]

    if envio=='Free Expedited Shipping':
        data.loc[0,['Envios_Free Standard Shipping', 'Envios_Minimum Order Free Shipping','Envios_Paid Shipping']]=[0,0,0]
    elif envio=='Free Standard Shipping':
        data.loc[0,['Envios_Free Standard Shipping', 'Envios_Minimum Order Free Shipping','Envios_Paid Shipping']]=[1,0,0]
    elif envio=='Minimum Order Free Shipping':
        data.loc[0,['Envios_Free Standard Shipping', 'Envios_Minimum Order Free Shipping','Envios_Paid Shipping']]=[0,1,0]
    else:
        data.loc[0,['Envios_Free Standard Shipping', 'Envios_Minimum Order Free Shipping','Envios_Paid Shipping']]=[0,0,1]
    
    if cat_prim=='Audio and accessories':
        data.loc[0,['Categoria Principal_Computer and accessories','Categoria Principal_Electronics',
       'Categoria Principal_Phones and accessories','Categoria Principal_TV and accessories',
       'Categoria Principal_Wireless Tech']]=[0,0,0,0,0]
    elif cat_prim=='Computer and accessories':
        data.loc[0,['Categoria Principal_Computer and accessories','Categoria Principal_Electronics',
       'Categoria Principal_Phones and accessories','Categoria Principal_TV and accessories',
       'Categoria Principal_Wireless Tech']]=[1,0,0,0,0]
    elif cat_prim=='Electronics':
        data.loc[0,['Categoria Principal_Computer and accessories','Categoria Principal_Electronics',
       'Categoria Principal_Phones and accessories','Categoria Principal_TV and accessories',
       'Categoria Principal_Wireless Tech']]=[0,1,0,0,0]
    elif cat_prim=='Phones and accessories':
        data.loc[0,['Categoria Principal_Computer and accessories','Categoria Principal_Electronics',
       'Categoria Principal_Phones and accessories','Categoria Principal_TV and accessories',
       'Categoria Principal_Wireless Tech']]=[0,0,1,0,0]
    elif cat_prim=='TV and accessories':
        data.loc[0,['Categoria Principal_Computer and accessories','Categoria Principal_Electronics',
       'Categoria Principal_Phones and accessories','Categoria Principal_TV and accessories',
       'Categoria Principal_Wireless Tech']]=[0,0,0,1,0]
    else:
        data.loc[0,['Categoria Principal_Computer and accessories','Categoria Principal_Electronics',
       'Categoria Principal_Phones and accessories','Categoria Principal_TV and accessories',
       'Categoria Principal_Wireless Tech']]=[0,0,0,0,1]
    
    if cat_sec=='Audio and accessories':
        data.loc[0,['secondary category_Audio and accessories','secondary category_Computer',
       'secondary category_Computer and accessories','secondary category_Digital camera and accessories',
       'secondary category_Discos duros', 'secondary category_Electronics',
       'secondary category_Home Theater',
       'secondary category_Phones and accessories',
       'secondary category_Smart-tech', 'secondary category_TV',
       'secondary category_TV and accessories',
       'secondary category_Wireless Tech']]=[1,0,0,0,0,0,0,0,0,0,0,0]
    elif cat_sec=='Computer':
        data.loc[0,['secondary category_Audio and accessories','secondary category_Computer',
       'secondary category_Computer and accessories','secondary category_Digital camera and accessories',
       'secondary category_Discos duros', 'secondary category_Electronics',
       'secondary category_Home Theater',
       'secondary category_Phones and accessories',
       'secondary category_Smart-tech', 'secondary category_TV',
       'secondary category_TV and accessories',
       'secondary category_Wireless Tech']]=[0,1,0,0,0,0,0,0,0,0,0,0]
    elif cat_sec=='Computer and accessories':
        data.loc[0,['secondary category_Audio and accessories','secondary category_Computer',
       'secondary category_Computer and accessories','secondary category_Digital camera and accessories',
       'secondary category_Discos duros', 'secondary category_Electronics',
       'secondary category_Home Theater',
       'secondary category_Phones and accessories',
       'secondary category_Smart-tech', 'secondary category_TV',
       'secondary category_TV and accessories',
       'secondary category_Wireless Tech']]=[0,0,1,0,0,0,0,0,0,0,0,0]
    elif cat_sec=='Digital camera and accessories':
        data.loc[0,['secondary category_Audio and accessories','secondary category_Computer',
       'secondary category_Computer and accessories','secondary category_Digital camera and accessories',
       'secondary category_Discos duros', 'secondary category_Electronics',
       'secondary category_Home Theater',
       'secondary category_Phones and accessories',
       'secondary category_Smart-tech', 'secondary category_TV',
       'secondary category_TV and accessories',
       'secondary category_Wireless Tech']]=[0,0,0,1,0,0,0,0,0,0,0,0]
    elif cat_sec=='Discos duros':
        data.loc[0,['secondary category_Audio and accessories','secondary category_Computer',
       'secondary category_Computer and accessories','secondary category_Digital camera and accessories',
       'secondary category_Discos duros', 'secondary category_Electronics',
       'secondary category_Home Theater',
       'secondary category_Phones and accessories',
       'secondary category_Smart-tech', 'secondary category_TV',
       'secondary category_TV and accessories',
       'secondary category_Wireless Tech']]=[0,0,0,0,1,0,0,0,0,0,0,0]
    elif cat_sec=='Electronics':
        data.loc[0,['secondary category_Audio and accessories','secondary category_Computer',
       'secondary category_Computer and accessories','secondary category_Digital camera and accessories',
       'secondary category_Discos duros', 'secondary category_Electronics',
       'secondary category_Home Theater',
       'secondary category_Phones and accessories',
       'secondary category_Smart-tech', 'secondary category_TV',
       'secondary category_TV and accessories',
       'secondary category_Wireless Tech']]=[0,0,0,0,0,1,0,0,0,0,0,0]
    elif cat_sec=='Home Theater':
        data.loc[0,['secondary category_Audio and accessories','secondary category_Computer',
       'secondary category_Computer and accessories','secondary category_Digital camera and accessories',
       'secondary category_Discos duros', 'secondary category_Electronics',
       'secondary category_Home Theater',
       'secondary category_Phones and accessories',
       'secondary category_Smart-tech', 'secondary category_TV',
       'secondary category_TV and accessories',
       'secondary category_Wireless Tech']]=[0,0,0,0,0,0,1,0,0,0,0,0]
    elif cat_sec=='Phones and accessories':
        data.loc[0,['secondary category_Audio and accessories','secondary category_Computer',
       'secondary category_Computer and accessories','secondary category_Digital camera and accessories',
       'secondary category_Discos duros', 'secondary category_Electronics',
       'secondary category_Home Theater',
       'secondary category_Phones and accessories',
       'secondary category_Smart-tech', 'secondary category_TV',
       'secondary category_TV and accessories',
       'secondary category_Wireless Tech']]=[0,0,0,0,0,0,0,1,0,0,0,0]
    elif cat_sec=='Smart-tech':
        data.loc[0,['secondary category_Audio and accessories','secondary category_Computer',
       'secondary category_Computer and accessories','secondary category_Digital camera and accessories',
       'secondary category_Discos duros', 'secondary category_Electronics',
       'secondary category_Home Theater',
       'secondary category_Phones and accessories',
       'secondary category_Smart-tech', 'secondary category_TV',
       'secondary category_TV and accessories',
       'secondary category_Wireless Tech']]=[0,0,0,0,0,0,0,0,1,0,0,0]
    elif cat_sec=='TV':
        data.loc[0,['secondary category_Audio and accessories','secondary category_Computer',
       'secondary category_Computer and accessories','secondary category_Digital camera and accessories',
       'secondary category_Discos duros', 'secondary category_Electronics',
       'secondary category_Home Theater',
       'secondary category_Phones and accessories',
       'secondary category_Smart-tech', 'secondary category_TV',
       'secondary category_TV and accessories',
       'secondary category_Wireless Tech']]=[0,0,0,0,0,0,0,0,0,1,0,0]
    elif cat_sec=='TV and accessories':
        data.loc[0,['secondary category_Audio and accessories','secondary category_Computer',
       'secondary category_Computer and accessories','secondary category_Digital camera and accessories',
       'secondary category_Discos duros', 'secondary category_Electronics',
       'secondary category_Home Theater',
       'secondary category_Phones and accessories',
       'secondary category_Smart-tech', 'secondary category_TV',
       'secondary category_TV and accessories',
       'secondary category_Wireless Tech']]=[0,0,0,0,0,0,0,0,0,0,1,0]
    elif cat_sec=='Wireless Tech':
        data.loc[0,['secondary category_Audio and accessories','secondary category_Computer',
       'secondary category_Computer and accessories','secondary category_Digital camera and accessories',
       'secondary category_Discos duros', 'secondary category_Electronics',
       'secondary category_Home Theater',
       'secondary category_Phones and accessories',
       'secondary category_Smart-tech', 'secondary category_TV',
       'secondary category_TV and accessories',
       'secondary category_Wireless Tech']]=[0,0,0,0,0,0,0,0,0,0,0,1]
    else:
        data.loc[0,['secondary category_Audio and accessories','secondary category_Computer',
       'secondary category_Computer and accessories','secondary category_Digital camera and accessories',
       'secondary category_Discos duros', 'secondary category_Electronics',
       'secondary category_Home Theater',
       'secondary category_Phones and accessories',
       'secondary category_Smart-tech', 'secondary category_TV',
       'secondary category_TV and accessories',
       'secondary category_Wireless Tech']]=[0,0,0,0,0,0,0,0,0,0,0,0]
                    
    data = data.astype(float)
    X=data.values
    
    modelo=load_model('digital_class.h5')  
    predictions = modelo.predict(X)
    
    digito_predicho = np.argmax(predictions)

    if digito_predicho==0:
        tienda='Bestbuy'
    elif digito_predicho==1:
        tienda='Walmart'
    elif digito_predicho==2:
        tienda='Bhphotovideo'
    else:
        tienda='Ebay'


    return tienda

prediction=prediccion_tienda(max_price,min_price,disponibilidad,estado,sale,envio,cat_prim,cat_sec)

prediction

1/1 [==============================] - 0s 45ms/step


'Walmart'